In [1]:
import random
import time
from threading import Thread

In [2]:
OWN_TRUCK_ID = 4
BROKEN_TRUCK_ID = 2
AMOUNT_OF_TRUCKS = 10
AMOUNT_OF_CONVOYS = 1
TICK_TIME = 1

In [3]:
class Truck(object):
    def __init__(self, truckId: int, convoyPosition: int, convoyLeaderId: int, speed: float, broken: bool, convoy):
        self.truckId = truckId
        self.convoyPosition = convoyPosition
        self.convoyLeaderId = convoyLeaderId
        self.speed = speed
        self.broken = broken
        self.convoy = convoy
        self.accelerating = False
        self.breaking = False
    
    def alive(self):
        return not self.broken
    
    def toggleBroken(self):
        self.broken = not self.broken
    
    def getTruckId(self):
        return self.truckId
    
    def setConvoyPosition(self, convoyPosition: int):
        self.convoyPosition = convoyPosition
        
    def getConvoyPosition(self):
        return self.convoyPosition
    
    def setConvoyLeaderId(self, convoyLeaderId: int):
        self.convoyLeaderId = convoyLeaderId
        
    def getConvoyLeaderId(self):
        return self.convoyLeaderId
    
    def setSpeed(self, speed: float):
        self.speed = speed
        
    def getSpeed(self):
        return self.speed

    def setConvoy(self, convoy):
        self.convoy = convoy
        
    def getConvoy(self):
        return self.convoy

    def breakTruck(self, speed: float):
        self.speed = speed

    def accelerateTruck(self, speed: float):
        self.speed = speed
    
    def startPoll(self):
        #answer = {'pos': ('alive', 'truckId')}
        oldConvoyPosition = self.convoyPosition
        
        trucks = self.convoy.getTrucks()
        trucksAlive = {}
        for convoyPosition, truckThread in trucks:
            if convoyPosition <= self.convoyPosition:
                trucksAlive[convoyPosition] = (truckThread.getTruck().alive(), truckThread.getTruck().getTruckId())
        newConvoyPosition = self.__bully__(trucksAlive)
        self.convoyPosition = newConvoyPosition
        return oldConvoyPosition - newConvoyPosition
    
    def __bully__(self, trucksAlive):
        newPositions = []
        for position, (alive, truckId) in trucksAlive:
            if alive:
                newPositions.append(truckId)
        return newPositions.index(self.truckId)
        #self.truckId = newPositions.index(self.truckId)

    def tellTruckBehindToPrepareBreaking(self, convoyPosition: int, speed: float):
        print(f'truck with position: {convoyPosition} in convoy with id: {self.convoy.convoyId} prepares to break\n')
        positionTruckBehind = convoyPosition + 1
        self.convoy.getTrucks()[convoyPosition].getTruck().breaking = not self.convoy.getTrucks()[convoyPosition].getTruck().breaking
        if positionTruckBehind in self.convoy.getTrucks():
            print(f'telling truck with position: {positionTruckBehind} in convoy with id: {self.convoy.convoyId} to prepare breaking\n')    
            self.convoy.getTrucks()[convoyPosition].getTruck().tellTruckBehindToPrepareBreaking(positionTruckBehind, speed)
        elif not positionTruckBehind in self.convoy.getTrucks(): 
            print(f'last truck with position: {positionTruckBehind} in convoy with id: {self.convoy.convoyId} confirms preparation, breaks, and tells truck in front to break aswell\n')        
            self.convoy.getTrucks()[convoyPosition].getTruck().breaking = not self.breaking
            self.convoy.getTrucks()[convoyPosition].getTruck().breakTruck(speed)
            self.convoy.getTrucks()[convoyPosition].getTruck().tellTruckInFrontToBreak(convoyPosition - 1, speed)
        else:
             print('Unexpected Exception')

    def tellTruckInFrontToBreak(self, convoyPosition: int, speed: float):
        print(f'truck with position: {convoyPosition} in convoy with id: {self.convoy.convoyId} breaks \n')
        positionTruckInFront = convoyPosition - 1
        self.convoy.getTrucks()[convoyPosition].getTruck().breakTruck(speed)
        self.convoy.getTrucks()[convoyPosition].getTruck().breaking = not self.convoy.getTrucks()[convoyPosition].getTruck().breaking
        if positionTruckInFront in self.convoy.getTrucks():
            print(f'truck with position: {positionTruckInFront} in convoy with id: {self.convoy.convoyId} tells the truck in front to break aswell\n')
            convoyPosition
            self.convoy.getTrucks()[convoyPosition].getTruck().tellTruckInFrontToBreak(positionTruckInFront, speed)
        elif not positionTruckInFront in self.convoy.getTrucks():
            print(f'convoy with id: {self.convoy.convoyId} done breaking\n\n')
            self.convoy.printConvoyTrucks()
        else:
            print('Unexpected Exception')
    
    def speedUpAndTellTruckBehindToAccelerate(self, convoyPosition: int, speed: float):
        print(f'truck with position: {convoyPosition} in convoy with id: {self.convoy.convoyId} accelerates\n')
        positionTruckBehind = convoyPosition + 1
        self.convoy.getTrucks()[convoyPosition].getTruck().accelerating = not self.convoy.getTrucks()[convoyPosition].getTruck().accelerating
        self.convoy.getTrucks()[convoyPosition].getTruck().accelerateTruck(speed)
        if positionTruckBehind in self.convoy.getTrucks():
            print(f'telling truck with position: {positionTruckBehind} in convoy with id: {self.convoy.convoyId} to accelerate\n')
            self.convoy.getTrucks()[convoyPosition].getTruck().speedUpAndTellTruckBehindToAccelerate(positionTruckBehind, speed)
        elif not positionTruckBehind in self.convoy.getTrucks():
            print(f'telling truck with position: {positionTruckBehind} in convoy with id: {self.convoy.convoyId} last truck in line has accelerated\n')
            self.convoy.getTrucks()[convoyPosition].getTruck().accelerating = not self.convoy.getTrucks()[convoyPosition].getTruck().accelerating
            self.convoy.getTrucks()[convoyPosition].getTruck().confirmAccelerationToTruckInFront(convoyPosition - 1, speed)


    def confirmAccelerationToTruckInFront(self, convoyPosition: int, speed: float):
        print(f'truck with position: {convoyPosition} in convoy with id: {self.convoy.convoyId} recieved acceleration validation\n')
        positionTruckInFront = convoyPosition - 1
        self.convoy.getTrucks()[convoyPosition].getTruck().accelerating = not self.convoy.getTrucks()[convoyPosition].getTruck().accelerating
        if positionTruckInFront in self.convoy.getTrucks():
            print(f'telling truck with position: {positionTruckInFront} in convoy with id: {self.convoy.convoyId} acceleration process is done\n')
            self.convoy.getTrucks()[convoyPosition].getTruck().confirmAccelerationToTruckInFront(positionTruckInFront, speed)
        elif not positionTruckInFront in self.convoy.getTrucks():
            print(f'convoy with id: {self.convoy.convoyId} done accelerating\n\n')
            self.convoy.printConvoyTrucks()
        else:
            print('Unexpected Exception')

In [4]:
class TruckThread(Thread):
    def __init__(self, truck):
        Thread.__init__(self)
        self.truck = truck
        self.arrived = False
        self.setName(f'truckThread_{self.truck.getTruckId()}')
    
    def run(self):
        #print(f'start engine of truck with id: {self.truck.getTruckId()} in convoy with id: {self.truck.getConvoy().getId()}\n\n')
        #self.convoy.determineNewConvoyLeader()
        #or
        #self.convoy.triggerElectionOnTruck(truckId)
        while(not self.arrived):
            pass
        
    def stop(self):
        print(f'stopping engine of truck with id: {self.truck.getTruckId()} in convoy with id: {self.truck.getConvoy().getId()}\n\n')

    def getTruck(self):
        return self.truck
    

In [5]:
class Convoy(object):
    def __init__(self, convoyId: int, amountOfTrucks: int):
        self.convoyId = convoyId
        self.truckIds = []
        self.trucks = {}
        self.__createTrucks__(amountOfTrucks)
    
    def __createTrucks__(self, amountOfTrucks: int):
        convoyLeaderId = 0
        speed = 0
        broken= False
        for convoyPosition in range(amountOfTrucks):
            truckId = self.__randomTruckId__()
            newTruck = Truck(truckId, convoyPosition, convoyLeaderId, speed, broken, self)
            newTruckThread = TruckThread(newTruck)
            self.trucks[convoyPosition] = newTruckThread
    
    def __randomTruckId__(self):
        success = False
        while not success:
            newId = random.randint(1, 1000)
            if not newId in self.truckIds:
                return newId
    def getRandomUniqueNewTruckId(self):
        newId = self.__randomTruckId__()
        return newId

    def getId(self):
        return self.convoyId

    def getTruckAt(self, convoyPosition: int): # Das ist dann nachher die Netzwerkadresse des Trucks
        return self.trucks[convoyPosition].getTruck()
    
    def getTrucks(self):
        return self.trucks
    
    def de_activateRandomTruck(self):
        amount = self.getAmountOfTrucks()
        rand = random.randint(0, amount-1)
        self.trucks[rand].toggleBroken()
    
    def getAmountOfTrucks(self):
        return len(self.trucks)
    
    def geta(self):
        return self.trucks[9].getTruck().startPoll()

    def printConvoyTrucks(self):
        for key, val in self.trucks.items():
            print(f'Number: {key}\nTruck: {val.getTruck().getTruckId()}\nSpeed: {val.getTruck().getSpeed()}\n\n')
    
    def tellLeaderOfConvoyToInitiateDeceleration(self, speed: float):
        self.trucks[0].getTruck().tellTruckBehindToPrepareBreaking(0, speed)

    def tellLeaderOfConovyToInitiateAcceleration(self, speed: float):
        self.trucks[0].getTruck().speedUpAndTellTruckBehindToAccelerate(0, speed)
    
    def tellTruckToInitiateElection(self):
        pass

    def joinTruckToConvoy(self):
        truckId = self.getRandomUniqueNewTruckId()
        convoyPosition = self.getAmountOfTrucks()
        convoyLeaderId = self.getTrucks()[0].getTruck().getTruckId()
        speed = self.getTrucks()[0].getTruck().getSpeed()
        broken = False
        newTruck = Truck(truckId, convoyPosition, convoyLeaderId, speed, broken, self)
        newTruckThread = TruckThread(newTruck)
        self.trucks[convoyPosition] = newTruckThread
        self.trucks[convoyPosition].start()

    def dismissTruckFromConvoy(self, convoyPosition: int):
        self.getTrucks().pop(convoyPosition)

In [6]:
class ConvoyThread(Thread):
    def __init__(self, convoy):
        Thread.__init__(self)
        self.convoy = convoy
        self.arrived = False
        self.setName(f'convoyThread_{self.convoy.getId()}')
    
    def run(self):
        print(f'starting trucks of convoy with id: {self.convoy.getId()}\n\n')
        for convoyId, truckThread in self.convoy.getTrucks().items():
            truckThread.getTruck().breakTruck(0)
            truckThread.start()
            #truckThread.join()

        while(not self.arrived):
            pass
        
    def stop(self):
        for convoyId, truckThread in self.convoy.getTrucks().items():
            truckThread.getTruck().breakTruck(0)
            truckThread.stop()
            print(f'stopped convoy with id: {self.convoy.getId()}\n\n')

    def getConvoy(self):
        return self.convoy

In [7]:
class ConvoyList(object):
    def __init__(self):
        self.convoys = {}
        self.createConvoys(AMOUNT_OF_CONVOYS)
    
    def getConvoys(self):
        return self.convoys

    def createConvoys(self, amountOfConvoys: int):
        print(f'marshalling {amountOfConvoys} convoy(s) with {AMOUNT_OF_TRUCKS} trucks each\n')
        for convoyId in range(amountOfConvoys):
            print(f'marshalling convoys with id {convoyId}\n')
            newConvoy = Convoy(convoyId, AMOUNT_OF_TRUCKS)
            newConvoyThread = ConvoyThread(newConvoy)
            self.convoys[convoyId] = newConvoyThread
            print(f'marshalling convoy with trucks:\n{newConvoy.printConvoyTrucks()}')

    def startEnginesOfAllConvoys(self, speed: float):
        for convoyId, convoyThread in self.convoys:
            print(f'starting engines of trucks in convoy with id: {convoyId}\n')
            convoyThread.start()
            #convoyThread.join()

    def startAllEnginesOfAConvoy(self, convoyId: int):
        print(f'starting engines of trucks in convoy with id: {convoyId}\n')
        self.convoys[convoyId].start()

    def accelerateConvoyInDictionary(self, convoyId: int, speed: float):
        print(f'accelerating trucks in convoy with id: {convoyId} by {speed} km/h\n')
        self.convoys[convoyId].getConvoy().tellLeaderOfConovyToInitiateAcceleration(speed)
    
    def decelerateConvoyInDictionary(self,convoyId: int, speed: float):
        self.convoys[convoyId].getConvoy().tellLeaderOfConvoyToInitiateDeceleration(speed)

    def stopEnginesOfAllConvoys(self):
        for convoyId, thread in self.convoys:
            print(f'stopping engines of all truck in convoy with id: {convoyId}')
            self.decelerateConvoyInDictionary(convoyId, 0)
            thread.stop()

    def stopAllEnginesOfAConvoy(self, convoyId: int):
        print(f'stopping engines of all truck in convoy with id: {convoyId}')
        self.decelerateConvoyInDictionary(convoyId, 0)
        self.convoys[convoyId].stop()

    def joinTruckToAConvoy(self, convoyId: int):
        print(f'joining truck to convoy with the id: {convoyId}')
        self.getConvoys()[convoyId].getConvoy().joinTruckToConvoy()
        self.getConvoys()[convoyId].getConvoy().printConvoyTrucks()
    
    def dismissTruckFromAConvoy(self, convoyId, convoyPosition):
        print(f'dismissing truck with position: {convoyPosition} form convoy with id: {convoyId}\n\n')
        self.getConvoys()[convoyId].getConvoy().dismissTruckFromConvoy(convoyPosition)
        self.getConvoys()[convoyId].getConvoy().printConvoyTrucks()

    def initiatePollingOnAConvoy(self, convoyId: int):
        pass

    #break:      tell leader to break. starts breaking alert handing it to the next truck and so forth. last truck breaks and hands                  forward breaking command and so forth.
    #accelerate: leader accelerates and hands acceleration command to next truck and so forth.


In [8]:
convoyList = ConvoyList()
time.sleep(TICK_TIME)
convoyList.startAllEnginesOfAConvoy(0)
time.sleep(TICK_TIME)
convoyList.accelerateConvoyInDictionary(0, 20)
time.sleep(TICK_TIME)
convoyList.decelerateConvoyInDictionary(0, 15)
time.sleep(TICK_TIME)
convoyList.decelerateConvoyInDictionary(0, 0)
time.sleep(TICK_TIME)
convoyList.joinTruckToAConvoy(0)
time.sleep(TICK_TIME)
convoyList.joinTruckToAConvoy(0)
time.sleep(TICK_TIME)
convoyList.dismissTruckFromAConvoy(0, 3)
time.sleep(TICK_TIME)
#convoyList.initiatePollingTruckOnAConvoy(0)
time.sleep(TICK_TIME)
convoyList.stopAllEnginesOfAConvoy(0)

marshalling 1 convoy(s) with 10 trucks each

marshalling convoys with id 0

Number: 0
Truck: 927
Speed: 0


Number: 1
Truck: 192
Speed: 0


Number: 2
Truck: 680
Speed: 0


Number: 3
Truck: 889
Speed: 0


Number: 4
Truck: 421
Speed: 0


Number: 5
Truck: 960
Speed: 0


Number: 6
Truck: 168
Speed: 0


Number: 7
Truck: 743
Speed: 0


Number: 8
Truck: 649
Speed: 0


Number: 9
Truck: 31
Speed: 0


marshalling convoy with trucks:
None
starting engines of trucks in convoy with id: 0

starting trucks of convoy with id: 0


accelerating trucks in convoy with id: 0 by 20 km/h

truck with position: 0 in convoy with id: 0 accelerates

telling truck with position: 1 in convoy with id: 0 to accelerate

truck with position: 1 in convoy with id: 0 accelerates

telling truck with position: 2 in convoy with id: 0 to accelerate

truck with position: 2 in convoy with id: 0 accelerates

telling truck with position: 3 in convoy with id: 0 to accelerate

truck with position: 3 in convoy with id: 0 accelerates